In [1]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import requests
from io import BytesIO
from bs4 import BeautifulSoup

def search_image():
    query = entry.get()
    url = f"https://www.google.com/search?q={query}&source=lnms&tbm=isch"
    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the HTML content for image URLs
        soup = BeautifulSoup(response.text, 'html.parser')
        images = soup.find_all('img')[1:7]  # Get the first 6 images

        # Clear the previous images
        for widget in image_frame.winfo_children():
            widget.destroy()

        # Display the images
        for img in images:
            img_url = img['src']
            img_response = requests.get(img_url)
            img_data = img_response.content
            image = Image.open(BytesIO(img_data))
            image.thumbnail((1000, 1000))  # Resize for better display
            img_tk = ImageTk.PhotoImage(image)
            img_label = ttk.Label(image_frame, image=img_tk)
            img_label.image = img_tk
            img_label.pack(side='left', padx=10, pady=10)
    else:
        error_label.config(text="Failed to retrieve images")

# Create the GUI window
root = tk.Tk()
root.title("Image Search")
root.geometry("1200x800")  # Set the window size to medium

# Create and place the entry box
entry = ttk.Entry(root, width=50)
entry.pack(pady=20)

# Create and place the search button
search_button = ttk.Button(root, text="Search", command=search_image)
search_button.pack(pady=20)

# Create a frame to hold the images
image_frame = ttk.Frame(root)
image_frame.pack(pady=20)

# Create and place the error label
error_label = ttk.Label(root, text="")
error_label.pack(pady=20)

# Start the Tkinter event loop
root.mainloop()


In [ ]:
import http.server
import socketserver
import urllib.parse
import json
from PIL import Image
import requests
from io import BytesIO
from bs4 import BeautifulSoup
import base64

PORT = 8000

class Handler(http.server.SimpleHTTPRequestHandler):
    def do_GET(self):
        if self.path.startswith('/search'):
            query = urllib.parse.parse_qs(urllib.parse.urlparse(self.path).query).get('query', [''])[0]
            url = f"https://www.google.com/search?q={query}&source=lnms&tbm=isch"
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            images_data = []

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                images = soup.find_all('img')[1:7]

                for img in images:
                    img_url = img['src']
                    img_response = requests.get(img_url)
                    img_data = img_response.content
                    image = Image.open(BytesIO(img_data))
                    image.thumbnail((500, 500))
                    buffered = BytesIO()
                    image.save(buffered, format="JPEG")
                    img_str = base64.b64encode(buffered.getvalue()).decode()
                    images_data.append(f"data:image/jpeg;base64,{img_str}")

            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(json.dumps(images_data).encode())

        else:
            super().do_GET()

httpd = socketserver.TCPServer(("", PORT), Handler)
print(f"Serving on port {PORT}")
httpd.serve_forever()


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Image Search</title>
    <style>
        body {
            font-family: Arial, sans-serif;
        }
        .search-container {
            margin: 20px;
        }
        .search-input {
            width: 300px;
            padding: 10px;
        }
        .search-button {
            padding: 10px;
        }
        .images-container {
            display: flex;
            flex-wrap: wrap;
        }
        .image-item {
            margin: 10px;
        }
        .image-item img {
            max-width: 100%;
            max-height: 150px;
        }
    </style>
</head>
<body>

    <div class="search-container">
        <input type="text" id="search-query" class="search-input" placeholder="Search for images">
        <button onclick="searchImages()" class="search-button">Search</button>
    </div>

    <div id="images-container" class="images-container"></div>

    <script>
        async function searchImages() {
            const query = document.getElementById('search-query').value;
            const response = await fetch(`/search?query=${encodeURIComponent(query)}`);
            const images = await response.json();

            const imagesContainer = document.getElementById('images-container');
            imagesContainer.innerHTML = '';

            images.forEach(src => {
                const img = document.createElement('img');
                img.src = src;
                const div = document.createElement('div');
                div.className = 'image-item';
                div.appendChild(img);
                imagesContainer.appendChild(div);
            });
        }
    </script>

</body>
</html>
